In [5]:
import pandas as pd
import numpy as np
import string
import Levenshtein as lev

# Punto 1. Limpieza y transformación de los datos

Utilizaré la técnica de tokenización manual como primer proceso de limpieza consistente en 6 pasos:

1. Carga de datos 
2. Revisión de la consistencia de los datos y transformación a string
3. Normalización de mayúsculas y minúsculas
4. Eliminación de puntuación y caracteres especiales
5. Separar por espacios
6. Eliminación de palabras con poco significado.
7. Definición de grupos y reclasificación
8. Guardado de base de datos en un archivo

### 1. Carga de base datos

In [205]:
dataSource = "data/DataAnalyst_YoFio_PT_ocupaciones.csv"
data=pd.read_csv(dataSource)

### 2. Revisión de la consistencia de la base de datos

In [206]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Empleos  560 non-null    object
dtypes: object(1)
memory usage: 4.5+ KB


In [207]:
data=data.astype("string")

### 3. Normalización a minúsculas

In [208]:
data.Empleos=data.Empleos.apply(lambda x: x.lower())

### 4. Eliminación de puntuación y caracteres especiales

In [209]:
caracteresEspeciales=str.maketrans('áéíóú','aeiou',string.punctuation)
data.Empleos = data.Empleos.apply(lambda empleo: empleo.translate(caracteresEspeciales))

### 5. Separar por espacios

In [210]:
data.Empleos=data.Empleos.apply(str.split)

### 6. Eliminación de palabras con poco significado

In [211]:
preposiciones = ['a', 'ante', 'bajo', 'cabe', 'con', 'contra', 'de', 'desde', 'en', 'entre', 'hacia', 'hasta', 'para', 'por', 'segun', 'sin', 'so', 'sobre', 'tras', 'durante', 'mediante', 'versus','via']
conjunciones = ['y', 'e', 'ni', 'que', 'pero', 'mas', 'aunque', 'sino', 'siquiera', 'o', 'u', 'ora', 'sea', 'bien']
otras = ['soy', 'una', 'uno','la', 'el', 'los']
palabrasPocoSignificado=preposiciones + conjunciones + otras

In [212]:
def Eliminar_Palabras(listaDePalabras, palabrasParaEliminar=palabrasPocoSignificado):
    for indice, palabra in enumerate(listaDePalabras):
        if palabra in palabrasParaEliminar:
            listaDePalabras.pop(indice)

data.Empleos.apply(lambda empleos: Eliminar_Palabras(empleos));

### 7. Definición de grupos y reclasificación

In [213]:
grupos={'empleado': ['empleado','empleada','trabajador','obrero', 'tecnico', 'coordinador', 'operador', 'trabajo','supervisor',
                    'servidor', 'ayudante', 'auxiliar', 'mesero', 'promotor', 'despachador', 'demostrador',
                    'maniobrista','verificador','encargado','analista','sectretaria', 'montacarguista'
                    'instalador','oficina','fabrica','soporte','comisionista','montacarguista','maquilador','conserje',
                    'almacenista','telemarketer','telemarketing','callcenter','hoteleria','recepcionista',
                    'mantenimiento','intencencia','limpieza','aseo','reclutador','logistica','cuidador','reclutamiento',
                    'marketing','hostest','cafeteria','carpinteria','rosticeria','capturista','gasolinera','ajustador',
                    'autolavado','servicio','call','center','camarista','monirotista','aplicador','repartidor','paqueteria',
                    'cajero','mensajero','agente','checador','aseador','rh','rrhh','coordinacion'],
        'vendedor': ['vendedor', 'ventas', 'venta'], 
        'chofer':['chofer','uber', 'didi', 'taxi', 'taxsista', 'conductor','transportista','transporte','bicitaxi',
                 'trailero'],
        'salud': ['doctor','medico','enfermera','cirgujano','dentista','odontologo','camillero','radiologo',
                  'terapeuta','trasladista','clinico','preventista'],
        'comerciante':['comerciante','comercio', 'vendo'],
        'hogar':['hogar','ama', 'casa'],
        'oficio':['oficio','mecanico','electricista','albañil','vidriero','reposteria','carrocero','electrico', 'carnicero',
                 'hojalatero', 'cocinero', 'chef', 'zapatero','panadero','pizzero','estilista','maquillista',
                 'albañileria','herreria','tapicero','jardinero','costurero','sastre','construccion','repartidor',
                 'taller','pirotecnia','verdulero','estetica','fotografo','fotografia','manualidades','tatuador',
                 'reparacion','reparador','pastelero','repostero','afinador','banquetero','soldador','ebanista',
                 'entenador','automotriz','taquero'],
        'profesionista':['ingeniero', 'abogado','contador', 'licenciado','contabilidad', 'arquitecto', 'veterinario',
                        'metrologo','auditor','programador','contratista','mercadologo','profesionisnta','diseñador',
                        'diseño'],
        'ejecutivo':['ejecutivo','gerente', 'administrador', 'finanzas','jefe','administrativo','administracion'],
        'empresario':['empresario','tienda','negocio','locatario','empresa'],
        'seguridad':['seguridad', 'guardia', 'policia','custodia','vigilante','velador'],
        'jubilado':['jubilado', 'pensionado'],
        'independiente':['independiente','freelance','emprendedor'],
        'docente':['docente','profesor','maestro', 'instructor'],
        'consultor':['consultor', 'asesor'],
        'desempleado':['desempleado']
       }

In [214]:
def Clasificador_Empleo(listaPalabras, diccionarioGrupos=grupos, precision=3):
    for palabra in listaPalabras:
        for grupo in list(diccionarioGrupos.keys()):
            for elemento in diccionarioGrupos[grupo]:
                distancia=lev.distance(palabra,elemento)
                if distancia < precision:
                    return grupo
    return 'otro' #str(listaPalabras)

In [215]:
data['EmpleoClasificado']=''
data.EmpleoClasificado=data.Empleos.apply(lambda empleo: Clasificador_Empleo(empleo))
data=data.drop(['Empleos'], axis =1)
data.rename(columns={'EmpleoClasificado':'Empleos'})

### 8. Guardado de base de datos en un archivo

In [218]:
data.to_csv('data/Resultado_Clasificacion_ocupaciones.csv')